Possible improvements:

- Use the crosscheck table to merge DR2 and DR3

- Check the mass: some stars do not have mass information? 


note: adding the last constrint "ap.mass_flame IS NOT NULL" decreases the number of results from 408 to 218



In [1]:
from astroquery.gaia import Gaia
# Gaia.MAIN_GAIA_TABLE = "gaiadr2.gaia_source"  # Select Data Release 2
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"  # Reselect Data Release 3, default

import numpy as np
import matplotlib.pyplot as plt
from astroquery.simbad import Simbad
import pandas as pd

from stellar_functions import *
from query import *



### Skip testing 1. Qeury Gaia DR2/3 data

names: https://gaia.aip.de/metadata/gaiadr3/gaia_source/

In [267]:
import pandas as pd
from astroquery.gaia import Gaia

# Query for the supplementary table
query_supp = """
SELECT gs.source_id, gs.ra, gs.dec, gs.phot_g_mean_mag, gs.phot_bp_mean_mag, gs.phot_rp_mean_mag, gs.bp_rp, gs.parallax, ap.teff_gspphot_marcs, ap.mass_flame_spec, ap.lum_flame_spec, ap.radius_flame_spec
FROM gaiadr3.gaia_source AS gs
JOIN gaiadr3.astrophysical_parameters_supp AS ap ON gs.source_id = ap.source_id
WHERE gs.phot_g_mean_mag < 12 
  AND gs.dec BETWEEN -90 AND 60
  AND gs.parallax >= 10
  AND (gs.duplicated_source = 'false' OR gs.duplicated_source IS NULL)
"""

# Execute the supplementary table query
job_supp = Gaia.launch_job_async(query_supp)
results_supp = job_supp.get_results().to_pandas()

# Query for the main table
query_main = """
SELECT gs.source_id, gs.ra, gs.dec, gs.phot_g_mean_mag, gs.phot_bp_mean_mag, gs.phot_rp_mean_mag, gs.bp_rp, gs.parallax, gs.teff_gspphot, ap.mass_flame, ap.lum_flame, ap.radius_flame, ap.spectraltype_esphs
FROM gaiadr3.gaia_source AS gs
JOIN gaiadr3.astrophysical_parameters AS ap ON gs.source_id = ap.source_id
WHERE gs.phot_g_mean_mag < 12 
  AND gs.dec BETWEEN -90 AND 60
  AND gs.parallax >= 10
  AND (gs.duplicated_source = 'false' OR gs.duplicated_source IS NULL)
"""

# Execute the main table query
job_main = Gaia.launch_job_async(query_main)
results_main = job_main.get_results().to_pandas()

# Convert source_id columns to strings
results_main['source_id'] = results_main['source_id'].astype(str)
results_supp['source_id'] = results_supp['source_id'].astype(str)

# Select only unique columns from each dataset
results_supp_unique = results_supp[['source_id', 'teff_gspphot_marcs', 'mass_flame_spec', 'lum_flame_spec', 'radius_flame_spec']]
results_main_unique = results_main[['source_id', 'ra', 'dec', 'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 'bp_rp', 'parallax', 'teff_gspphot', 'mass_flame', 'lum_flame', 'radius_flame', 'spectraltype_esphs']]

# Merge the results, prioritizing the supplementary table
results_merged = pd.merge(
    results_main_unique,
    results_supp_unique,
    on='source_id',
    how='outer',
    suffixes=('', '_supp')
)

# Fill missing values from the main table with values from the supplementary table
results_merged['teff_gspphot'] = results_merged['teff_gspphot_marcs'].combine_first(results_merged['teff_gspphot'])
results_merged['mass'] = results_merged['mass_flame'].combine_first(results_merged['mass_flame_spec'])
results_merged['luminosity'] = results_merged['lum_flame'].combine_first(results_merged['lum_flame_spec'])
results_merged['radius'] = results_merged['radius_flame'].combine_first(results_merged['radius_flame_spec'])

# Drop the individual columns if needed
results_final = results_merged.drop(columns=['teff_gspphot', 'mass_flame', 'lum_flame', 'radius_flame', 'mass_flame_spec', 'lum_flame_spec', 'radius_flame_spec'])

# Display the final results
print(results_final)

# Save the results
directory = '../results/'
results_supp.to_excel(directory+'results_supp.xlsx', index=False)
results_main.to_excel(directory+'results_main.xlsx', index=False)
results_final.to_excel(directory+'results_final.xlsx', index=False)


adjust_column_widths(directory+'results_supp.xlsx')
adjust_column_widths(directory+'results_main.xlsx')
adjust_column_widths(directory+'results_final.xlsx')


                source_id          ra        dec  phot_g_mean_mag  \
0     5937183024214130304  250.185079 -51.477674         6.224640   
1     5912398348432135552  262.774129 -60.684284         3.596028   
2     5912921509809469952  264.635582 -58.544378        11.327970   
3     5912928381756917888  264.019142 -58.666138        11.191809   
4     5912380962403300096  263.521562 -59.777571         7.459463   
...                   ...         ...        ...              ...   
2169  6176768149719900416         NaN        NaN              NaN   
2170  6175834462486641920         NaN        NaN              NaN   
2171  6173420003671144704         NaN        NaN              NaN   
2172  6175893556940258944         NaN        NaN              NaN   
2173  6175893561236253056         NaN        NaN              NaN   

      phot_bp_mean_mag  phot_rp_mean_mag     bp_rp   parallax  \
0             6.379528          5.939466  0.440062  19.588654   
1             3.600711          3.627106 

The code below does exactly the same as the one above

DR3

In [35]:
# Query for the supplementary table
query_combined = """
SELECT gs.source_id AS dr3_source_id, 
       gs.ra, 
       gs.dec, 
       gs.phot_g_mean_mag, 
       gs.phot_bp_mean_mag, 
       gs.phot_rp_mean_mag, 
       gs.bp_rp, 
       gs.parallax,
       COALESCE(ap_supp.teff_gspphot_marcs, ap.teff_gspphot, gs.teff_gspphot) AS teff_gspphot,
       COALESCE(ap_supp.mass_flame_spec, ap.mass_flame) AS mass,
       COALESCE(ap_supp.lum_flame_spec, ap.lum_flame) AS luminosity,
       COALESCE(ap_supp.radius_flame_spec, ap.radius_flame) AS radius,
       ap.spectraltype_esphs
FROM gaiadr3.gaia_source AS gs
LEFT JOIN gaiadr3.astrophysical_parameters_supp AS ap_supp 
    ON gs.source_id = ap_supp.source_id
LEFT JOIN gaiadr3.astrophysical_parameters AS ap 
    ON gs.source_id = ap.source_id
WHERE gs.phot_g_mean_mag < 9
  AND gs.dec BETWEEN -90 AND 60
  AND gs.parallax >= 50
  AND (gs.duplicated_source = 'false' OR gs.duplicated_source IS NULL)
"""

# Execute the main table query
job_combined = Gaia.launch_job_async(query_combined)
dr3 = job_combined.get_results().to_pandas()
dr3['dr3_source_id'] = dr3['dr3_source_id'].astype(str)

# Rename the columns to match the desired headers
dr3 = dr3.rename(columns={
    'ra': 'RA',
    'dec': 'DEC',
    'phot_g_mean_mag': 'Phot G Mean Mag',
    'phot_bp_mean_mag': 'Phot BP Mean Mag',
    'phot_rp_mean_mag': 'Phot RP Mean Mag',
    'bp_rp': 'BP-RP',
    'parallax': 'Parallax',
    'teff_gspphot': 'T_eff [K]',
    'mass': 'Mass [M_Sun]',
    'luminosity': 'Luminosity [L_Sun]',
    'radius': 'Radius [R_Sun]',
    'spectraltype_esphs': 'Spectral Type'
})

# Save the updated dataframe to Excel
directory = '../results/'
filename = directory + 'results_dr3.xlsx'
dr3.to_excel(filename, index=False)
adjust_column_widths(filename)

# Display the first few rows of the dataframe
display(dr3.head())
len(dr3)

INFO: Query finished. [astroquery.utils.tap.core]


,dr3_source_id,RA,DEC,Phot G Mean Mag,Phot BP Mean Mag,Phot RP Mean Mag,BP-RP,Parallax,T_eff [K],Mass [M_Sun],Luminosity [L_Sun],Radius [R_Sun],Spectral Type
0,4683897617110115200,6.482505,-77.252795,2.680732,3.295626,2.371489,0.924137,133.719406,,,,,F
1,5698015743046182272,121.885630,-24.304116,2.750589,3.302004,2.512174,0.789830,51.399886,,,,,F
2,5826168461855385472,238.783806,-63.432511,2.784391,3.186963,2.558162,0.628800,80.485412,,,,,A
3,6838311796136238976,326.761392,-16.128607,2.836813,3.290359,2.603107,0.687251,85.940560,,,,,B
4,4269932382607207040,275.325059,-2.901943,2.990450,3.710574,2.473570,1.237004,52.441308,,,,,G


457

DR2 
use JOIN, because it seems when source_id2... <NA>

In [34]:
# Query for the supplementary table
'''
dr2['source_id'] are identical to dr2['dr2_source_id']
'''

query_combined = """
SELECT gs.source_id AS dr2_source_id, crossmatch.dr3_source_id,
       gs.ra, 
       gs.dec, 
       gs.phot_g_mean_mag, 
       gs.phot_bp_mean_mag, 
       gs.phot_rp_mean_mag, 
       gs.bp_rp, 
       gs.parallax,
       COALESCE(ap_supp.teff_gspphot_marcs, ap.teff_gspphot, gs.teff_val) AS teff_gspphot,
       COALESCE(ap_supp.mass_flame_spec, ap.mass_flame) AS mass,
       COALESCE(ap_supp.lum_flame_spec, ap.lum_flame, gs.lum_val) AS luminosity,
       COALESCE(ap_supp.radius_flame_spec, ap.radius_flame, gs.radius_val) AS radius,
       ap.spectraltype_esphs
FROM gaiadr2.gaia_source AS gs
LEFT JOIN gaiadr3.dr2_neighbourhood AS crossmatch
    ON gs.source_id = crossmatch.dr2_source_id
LEFT JOIN gaiadr3.astrophysical_parameters_supp AS ap_supp 
    ON crossmatch.dr3_source_id = ap_supp.source_id
LEFT JOIN gaiadr3.astrophysical_parameters AS ap 
    ON crossmatch.dr3_source_id = ap.source_id
WHERE gs.phot_g_mean_mag < 9
  AND gs.dec BETWEEN -90 AND 60
  AND gs.parallax >= 50
  AND (gs.duplicated_source = 'false' OR gs.duplicated_source IS NULL)
  AND COALESCE(ap_supp.radius_flame_spec, ap.radius_flame) < 3
  AND COALESCE(ap_supp.lum_flame_spec, ap.lum_flame) < 10
  AND COALESCE(ap_supp.teff_gspphot_marcs, ap.teff_gspphot) < 7000  
"""

# Execute the main table query
job_combined = Gaia.launch_job_async(query_combined)
dr2 = job_combined.get_results().to_pandas()
dr2['dr2_source_id'] = dr2['dr2_source_id'].astype(str)
dr2['dr3_source_id'] = dr2['dr3_source_id'].astype(str)

# Rename the columns to match the desired headers
dr2 = dr2.rename(columns={
    'ra': 'RA',
    'dec': 'DEC',
    'phot_g_mean_mag': 'Phot G Mean Mag',
    'phot_bp_mean_mag': 'Phot BP Mean Mag',
    'phot_rp_mean_mag': 'Phot RP Mean Mag',
    'bp_rp': 'BP-RP',
    'parallax': 'Parallax',
    'teff_gspphot': 'T_eff [K]',
    'mass': 'Mass [M_Sun]',
    'luminosity': 'Luminosity [L_Sun]',
    'radius': 'Radius [R_Sun]',
    'spectraltype_esphs': 'Spectral Type'
})

# Save the updated dataframe to Excel
filename = directory + 'results_dr2.xlsx'
dr2.to_excel(filename, index=False)
adjust_column_widths(filename)

# Display the first few rows of the dataframe
display(dr2.head())
len(dr2)

INFO: Query finished. [astroquery.utils.tap.core]


,dr2_source_id,dr3_source_id,RA,DEC,Phot G Mean Mag,Phot BP Mean Mag,Phot RP Mean Mag,BP-RP,Parallax,T_eff [K],Mass [M_Sun],Luminosity [L_Sun],Radius [R_Sun],Spectral Type
0,4911306239828325760,4911306239828325760,24.950579,-56.196402,5.604027,6.110481,5.017446,1.093035,122.133341,4902.265,0.8265656,0.3073524,0.7482267,G
1,4911306239828325632,4911306239828325632,24.951252,-56.193253,5.481575,5.975546,4.918038,1.057508,122.055156,5044.24,0.848425,0.3307818,0.72947353,G
2,5640565607657897728,5640565607657897728,128.208941,-31.497569,6.150999,6.585626,5.606073,0.979553,82.090036,5207.114,0.87969726,0.39650527,0.7812532,G
3,3657653114880309248,3657653114880309248,210.259726,-2.652290,8.858808,9.961246,7.841961,2.119285,96.039794,3637.6152,0.527742,0.04382887,0.5488766,M
4,3359074685047632640,3359074685047632640,99.291543,17.566270,8.876039,9.875572,7.916457,1.959115,99.916379,3741.691,,0.040032268,0.53797454,M


288

dr2/dr3

In [45]:
# Query for the supplementary table
'''
dr2['source_id'] are identical to dr2['dr2_source_id']
'''

query_combined = """
SELECT gs2.source_id AS dr2_source_id, 
       gs3.source_id AS dr3_source_id, 
       crossmatch.dr3_source_id,
       COALESCE(gs3.ra, gs2.ra) AS ra,
       COALESCE(gs3.dec, gs2.dec) AS dec, 
       COALESCE(gs3.phot_g_mean_mag, gs2.phot_g_mean_mag) AS phot_g_mean_mag, 
       COALESCE(gs3.phot_bp_mean_mag, gs2.phot_bp_mean_mag) AS phot_bp_mean_mag, 
       COALESCE(gs3.phot_rp_mean_mag, gs2.phot_rp_mean_mag) AS phot_rp_mean_mag, 
       COALESCE(gs3.bp_rp, gs2.bp_rp) AS bp_rp, 
       COALESCE(gs3.parallax, gs2.parallax) AS parallax,
       COALESCE(ap_supp.teff_gspphot_marcs, ap.teff_gspphot, gs3.teff_gspphot, gs2.teff_val) AS teff_gspphot,
       COALESCE(ap_supp.mass_flame_spec, ap.mass_flame) AS mass,
       COALESCE(ap_supp.lum_flame_spec, ap.lum_flame, gs2.lum_val) AS luminosity,
       COALESCE(ap_supp.radius_flame_spec, ap.radius_flame, gs2.radius_val) AS radius,
       ap.spectraltype_esphs
FROM gaiadr3.gaia_source AS gs3 
FULL JOIN gaiadr3.dr2_neighbourhood AS crossmatch
    ON gs3.source_id = crossmatch.dr3_source_id
FULL JOIN gaiadr3.astrophysical_parameters_supp AS ap_supp 
    ON gs3.source_id = ap_supp.source_id
FULL JOIN gaiadr3.astrophysical_parameters AS ap 
    ON gs3.source_id = ap.source_id
FULL JOIN gaiadr2.gaia_source AS gs2
    ON crossmatch.dr2_source_id = gs2.source_id
WHERE COALESCE(gs3.phot_g_mean_mag, gs2.phot_g_mean_mag) < 9
  AND COALESCE(gs3.dec, gs2.dec) BETWEEN -90 AND 60
  AND COALESCE(gs3.parallax, gs2.parallax) >= 50
  AND (gs2.duplicated_source = 'false' OR gs2.duplicated_source IS NULL)
  AND (gs3.duplicated_source = 'false' OR gs3.duplicated_source IS NULL)
"""

# Execute the main table query
job_combined = Gaia.launch_job_async(query_combined)
dr2_dr3 = job_combined.get_results().to_pandas()
dr2_dr3['dr2_source_id'] = dr2_dr3['dr2_source_id'].astype(str)
dr2_dr3['dr3_source_id'] = dr2_dr3['dr3_source_id'].astype(str)

# Rename the columns to match the desired headers
dr2_dr3 = dr2_dr3.rename(columns={
    'ra': 'RA',
    'dec': 'DEC',
    'phot_g_mean_mag': 'Phot G Mean Mag',
    'phot_bp_mean_mag': 'Phot BP Mean Mag',
    'phot_rp_mean_mag': 'Phot RP Mean Mag',
    'bp_rp': 'BP-RP',
    'parallax': 'Parallax',
    'teff_gspphot': 'T_eff [K]',
    'mass': 'Mass [M_Sun]',
    'luminosity': 'Luminosity [L_Sun]',
    'radius': 'Radius [R_Sun]',
    'spectraltype_esphs': 'Spectral Type'
})

# Save the updated dataframe to Excel
filename = directory + 'results_dr2_dr3.xlsx'
dr2_dr3.to_excel(filename, index=False)
adjust_column_widths(filename)

# Display the first few rows of the dataframe
display(dr2_dr3.head())
len(dr2_dr3)

500 Error 500:
null


HTTPError: Error 500:
null

In [ ]:
SELECT gs2.source_id AS dr2_source_id, 
       gs3.source_id AS dr3_source_id, 
       crossmatch.dr3_source_id,
       COALESCE(gs3.ra, gs2.ra) AS ra,
       COALESCE(gs3.dec, gs2.dec) AS dec, 
       COALESCE(gs3.phot_g_mean_mag, gs2.phot_g_mean_mag) AS phot_g_mean_mag, 
       COALESCE(gs3.phot_bp_mean_mag, gs2.phot_bp_mean_mag) AS phot_bp_mean_mag, 
       COALESCE(gs3.phot_rp_mean_mag, gs2.phot_rp_mean_mag) AS phot_rp_mean_mag, 
       COALESCE(gs3.bp_rp, gs2.bp_rp) AS bp_rp, 
       COALESCE(gs3.parallax, gs2.parallax) AS parallax,
       COALESCE(ap_supp.teff_gspphot_marcs, ap.teff_gspphot, gs3.teff_gspphot, gs2.teff_val) AS teff_gspphot,
       COALESCE(ap_supp.mass_flame_spec, ap.mass_flame) AS mass,
       COALESCE(ap_supp.lum_flame_spec, ap.lum_flame, gs2.lum_val) AS luminosity,
       COALESCE(ap_supp.radius_flame_spec, ap.radius_flame, gs2.radius_val) AS radius,
       ap.spectraltype_esphs
FROM gaiadr3.gaia_source AS gs3 
FULL JOIN gaiadr3.dr2_neighbourhood AS crossmatch
    ON gs3.source_id = crossmatch.dr3_source_id
FULL JOIN gaiadr3.astrophysical_parameters_supp AS ap_supp 
    ON gs3.source_id = ap_supp.source_id
FULL JOIN gaiadr3.astrophysical_parameters AS ap 
    ON gs3.source_id = ap.source_id
FULL JOIN gaiadr2.gaia_source AS gs2
    ON crossmatch.dr2_source_id = gs2.source_id
WHERE COALESCE(gs3.phot_g_mean_mag, gs2.phot_g_mean_mag) < 9
  AND COALESCE(gs3.dec, gs2.dec) BETWEEN -90 AND 60
  AND COALESCE(gs3.parallax, gs2.parallax) >= 50
  AND (gs2.duplicated_source = 'false' OR gs2.duplicated_source IS NULL)
  AND (gs3.duplicated_source = 'false' OR gs3.duplicated_source IS NULL)


In [23]:
dr2_source_ids = tuple(dr2['dr2_source_id'])

crossmatch_query = f"""
SELECT dr2_source_id, dr3_source_id
FROM gaiadr3.dr2_neighbourhood
WHERE dr2_source_id IN {dr2_source_ids}
"""

# Execute the crossmatch query
job_crossmatch = Gaia.launch_job_async(crossmatch_query)
crossmatch_results = job_crossmatch.get_results().to_pandas()
crossmatch_results['dr2_source_id'] = crossmatch_results['dr2_source_id'].astype(str)
crossmatch_results['dr3_source_id'] = crossmatch_results['dr3_source_id'].astype(str)
filename = directory + 'crossmatch_results.xlsx'
crossmatch_results.to_excel(filename, index=False)
adjust_column_widths(filename)
crossmatch_results


INFO: Query finished. [astroquery.utils.tap.core]


,dr2_source_id,dr3_source_id
0,25488745411919360,25488745411919360
1,53112944270520832,53112944270520832
2,70051608089857536,70051608089857536
3,96331172942614528,96331172942614528
4,145421309108301184,145421309108301184
...,...,...
293,6832674634380238848,6832674634380238848
294,6847167606385195648,6847167606385195648
295,6863535898551993472,6863535898551993472
296,6866310172545095424,6866310172545095424


In [185]:
import pandas as pd

# Merge dr2 and dr3 on 'dr3_source_id'
merged_df = pd.merge(dr3, dr2, on='dr3_source_id', suffixes=('_dr3', '_dr2'), how='outer')

# Create a new DataFrame with preferred values from dr3, falling back to dr2 if necessary
final_df = pd.DataFrame()

# Include 'dr2_source_id' and 'dr3_source_id' as the first two columns
final_df['dr2_source_id'] = merged_df['dr2_source_id']
final_df['dr3_source_id'] = merged_df['dr3_source_id']

# List of columns to merge, assuming dr3 and dr2 have the same set of columns
columns_to_merge = [
    'RA', 'DEC', 'Phot G Mean Mag', 'Phot BP Mean Mag', 'Phot RP Mean Mag', 'BP-RP',
    'Parallax', 'T_eff [K]', 'Mass [M_Sun]', 'Luminosity [L_Sun]',
    'Radius [R_Sun]', 'Spectral Type'
]

# Iterate over the columns to merge
for column in columns_to_merge:
    final_df[column] = merged_df[f'{column}_dr3'].combine_first(merged_df[f'{column}_dr2'])

# Save the final DataFrame to Excel
filename = directory + 'results_dr2_dr3.xlsx'
final_df.to_excel(filename, index=False)
adjust_column_widths(filename)

# Display the first few rows of the final DataFrame
display(final_df.head())
print(f"Number of entries in the final DataFrame: {len(final_df)}")


,dr2_source_id,dr3_source_id,RA,DEC,Phot G Mean Mag,Phot BP Mean Mag,Phot RP Mean Mag,BP-RP,Parallax,T_eff [K],Mass [M_Sun],Luminosity [L_Sun],Radius [R_Sun],Spectral Type
0,NaN,5912380962403300096,263.521562,-59.777571,7.459463,7.753087,6.992328,0.760759,26.695519,5826.763,1.0270743,1.0723715,1.0139297,G
1,NaN,1837182671876057472,304.555931,29.203753,8.813396,9.034739,7.764806,1.269933,29.773776,4862.6016,0.7625022,0.30370355,0.88247246,G
2,NaN,1837182676181120896,304.556314,29.203715,9.528364,9.299628,7.864410,1.435218,29.769599,5044.509,0.70530427,0.18704037,0.8509922,G
3,NaN,4471238083887731456,270.637685,4.417389,8.704765,9.150189,8.093346,1.056844,25.687915,5015.104,0.7943323,0.39840892,0.83611274,K
4,NaN,4471087175915086080,271.405998,4.655748,6.649316,6.953541,6.166453,0.787088,41.474060,5691.311,1.0329351,0.9404929,0.87734646,G


Number of entries in the final DataFrame: 6099


In [186]:
merged_df

,dr3_source_id,RA_dr3,DEC_dr3,Phot G Mean Mag_dr3,Phot BP Mean Mag_dr3,Phot RP Mean Mag_dr3,BP-RP_dr3,Parallax_dr3,T_eff [K]_dr3,Mass [M_Sun]_dr3,...,Phot G Mean Mag_dr2,Phot BP Mean Mag_dr2,Phot RP Mean Mag_dr2,BP-RP_dr2,Parallax_dr2,T_eff [K]_dr2,Mass [M_Sun]_dr2,Luminosity [L_Sun]_dr2,Radius [R_Sun]_dr2,Spectral Type_dr2
0,5912380962403300096,263.521562,-59.777571,7.459463,7.753087,6.992328,0.760759,26.695519,5826.763,1.0270743,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1837182671876057472,304.555931,29.203753,8.813396,9.034739,7.764806,1.269933,29.773776,4862.6016,0.7625022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1837182676181120896,304.556314,29.203715,9.528364,9.299628,7.864410,1.435218,29.769599,5044.509,0.70530427,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4471238083887731456,270.637685,4.417389,8.704765,9.150189,8.093346,1.056844,25.687915,5015.104,0.7943323,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4471087175915086080,271.405998,4.655748,6.649316,6.953541,6.166453,0.787088,41.474060,5691.311,1.0329351,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6094,5407414407508423680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.085078,8.485525,7.568351,0.917174,26.992809,5409.9487,0.8972189,0.59520715,0.8781961,G
6095,5407414407508442112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.250566,11.039318,9.417773,1.621545,26.910121,4153.005,0.6670827,0.12740755,0.6894917,K
6096,5114240352186781696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.269168,11.106816,9.401341,1.705475,35.218015,4021.2742,0.6024087,0.081752606,0.6964844,K
6097,4464207428577214464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.247616,7.659267,6.716437,0.942830,41.259192,5289.4365,0.9276119,0.54569334,0.8102098,G


In [188]:

merged_results = merged_df.copy()

#-------------------------------------------------------------
# Data Cleaning
#-------------------------------------------------------------

# ***** Step 1: Identify repeated dr2_source_id entries *****
non_empty_dr2 = merged_results[merged_results['dr2_source_id'].notna() & (merged_results['dr2_source_id'] != '')]
repeated_dr2_ids = non_empty_dr2[non_empty_dr2.duplicated('dr2_source_id', keep=False)]['dr2_source_id'].unique()
repeated_entries = merged_results[merged_results['dr2_source_id'].isin(repeated_dr2_ids)]

# Save repeated entries to Excel
repeated_entries.to_excel(directory + 'repeated_entries.xlsx', index=False)
adjust_column_widths(directory + 'repeated_entries.xlsx')

# ***** Step 2: Clean repeated entries *****
def check_dr3_availability(row):
    '''
    Function to check if any DR3 data is available for a given row.
    '''
    dr3_columns = [col for col in row.index if col.endswith('_dr3')]
    return not row[dr3_columns].isnull().all()

def process_repeated_group(group):
    '''
    Function to process a group of repeated entries with the same dr2_source_id.
    '''
    if len(group) != 2:
        # If there are more than 2 entries, keep only the first one
        return group.iloc[1:].index.tolist()  # Return indices of rows to remove
    
    row1, row2 = group.iloc[0], group.iloc[1]
    dr3_available1 = check_dr3_availability(row1)
    dr3_available2 = check_dr3_availability(row2)
    
    if not dr3_available1 and not dr3_available2:
        # If both rows have no DR3 data, remove the second one
        return [group.index[1]]
    elif dr3_available1 and not dr3_available2:
        # If only the first row has DR3 data, remove the second one
        return [group.index[1]]
    elif not dr3_available1 and dr3_available2:
        # If only the second row has DR3 data, remove the first one
        return [group.index[0]]
    else:
        # If both rows have DR3 data, remove the second one
        return [group.index[1]]

# Process repeated entries and get indices of rows to remove
rows_to_remove_indices = repeated_entries.groupby('dr2_source_id').apply(process_repeated_group).sum()

# Get the rows to be removed
rows_to_remove = repeated_entries.loc[rows_to_remove_indices]

# Remove the identified rows from merged_results
clean_merged_results = merged_results[~merged_results.index.isin(rows_to_remove.index)]

# Reset index of clean_merged_results
clean_merged_results = clean_merged_results.reset_index(drop=True)

# Print some information about the results
print(f"Original shape of merged_results: {merged_results.shape}")
print(f"Shape after removing duplicates: {clean_merged_results.shape}")
print(f"Number of rows removed: {merged_results.shape[0] - clean_merged_results.shape[0]}")

# Save clean_merged_results to Excel
clean_merged_results.to_excel(directory + 'clean_merged_results.xlsx', index=False)
adjust_column_widths(directory + 'clean_merged_results.xlsx')

# Save rows_to_remove to Excel
rows_to_remove.to_excel(directory + 'removed_rows.xlsx', index=False)
adjust_column_widths(directory + 'removed_rows.xlsx')

# Check if there are still any duplicates
remaining_duplicates = clean_merged_results[clean_merged_results.duplicated('dr2_source_id', keep=False)]
print(f"\nNumber of remaining duplicate dr2_source_id: {len(remaining_duplicates['dr2_source_id'].unique())}")

if not remaining_duplicates.empty:
    print("\nExample of remaining duplicates:")
    print(remaining_duplicates.groupby('dr2_source_id').first().head())
else:
    print("\nNo remaining duplicates found.")

# ***** Step 3: Consolidate the data *****
# Create a new DataFrame to store the final consolidated data
df = clean_merged_results.copy()

# Function to choose between DR3 and DR2 values
def choose_value(row, col_name):
    dr3_col = f'{col_name}_dr3'
    dr2_col = f'{col_name}_dr2'
    return row[dr3_col] if pd.notnull(row[dr3_col]) else row[dr2_col]

# List of columns to process
columns_to_process = ['RA', 'DEC', 'Phot G Mean Mag', 'Phot BP Mean Mag', 'Phot RP Mean Mag', 'BP-RP',
    'Parallax', 'T_eff [K]', 'Mass [M_Sun]', 'Luminosity [L_Sun]',
    'Radius [R_Sun]', 'Spectral Type']

# Process each column
for col in columns_to_process:
    df[col] = df.apply(lambda row: choose_value(row, col), axis=1)

# # For other columns
# other_columns = ['mass_flame', 'lum_flame', 'radius_flame', 'spectraltype_esphs']
# for col in other_columns:
#     df[col] = df.apply(lambda row: choose_value(row, col), axis=1)

# # Add bp_rp column from DR3 if available
# df['bp_rp'] = df['bp_rp'].fillna(df['phot_bp_mean_mag'] - df['phot_rp_mean_mag'])

# # Add the new source_id column
# df['source_id'] = df['source_id_dr3'].fillna(df['source_id_dr2'])

# Update the final columns list to include the new source_id column
final_columns = ['dr2_source_id', 'dr3_source_id', 'RA', 'DEC', 'Phot G Mean Mag', 'Phot BP Mean Mag', 'Phot RP Mean Mag', 'BP-RP',
    'Parallax', 'T_eff [K]', 'Mass [M_Sun]', 'Luminosity [L_Sun]',
    'Radius [R_Sun]', 'Spectral Type']

# Create the final dataframe
df_consolidated = df[final_columns]



# Create a new DataFrame instead of modifying the existing one
new_columns = ['source_id', 'HD Number', 'GJ Number', 'HIP Number', 'Object Type']
df_new = pd.DataFrame(columns=new_columns)

# Populate the new DataFrame
df_new['source_id'] = df_consolidated['dr3_source_id'].fillna(df_consolidated['dr2_source_id'])



# Use the retry function
for index, row in df_new.iterrows():
    simbad_info = get_simbad_info_with_retry(row['source_id'])
    if simbad_info:
        df_new.loc[index, 'HD Number'] = simbad_info['HD Number']
        df_new.loc[index, 'GJ Number'] = simbad_info['GJ Number']
        df_new.loc[index, 'HIP Number'] = simbad_info['HIP Number']
        df_new.loc[index, 'Object Type'] = simbad_info['Object Type']











# Combine the new DataFrame with the original one
df_consolidated = pd.concat([df_consolidated, df_new[['HD Number', 'GJ Number', 'HIP Number', 'Object Type']]], axis=1)

# Update the final columns list to include the new columns
final_columns.extend(['HD Number', 'GJ Number', 'HIP Number', 'Object Type'])

# Create the final dataframe with the updated column list
df_consolidated = df_consolidated[final_columns]

# Rename the columns
# df_consolidated = df_consolidated.rename(columns={
#     'mass_flame': 'Mass [M_Sun]',
#     'lum_flame': 'Luminosity [L_Sun]',
#     'radius_flame': 'Radius [R_Sun]',
#     'phot_g_mean_mag': 'Phot G Mean Mag',
#     'phot_bp_mean_mag': 'Phot BP Mean Mag',
#     'phot_rp_mean_mag': 'Phot RP Mean Mag',
#     'bp_rp': 'BP-RP',
#     'parallax': 'Parallax',
#     'ra': 'RA',
#     'dec': 'DEC',
#     'spectraltype_esphs': 'Spectral Type'
# })

# Save the result to a new Excel file
df_consolidated.to_excel(directory + 'consolidated_results.xlsx', index=False)
adjust_column_widths(directory + 'consolidated_results.xlsx')

# Display some statistics
print(f"Total number of stars: {len(df_consolidated)}")
print(f"Number of stars with DR3 source_id: {df_consolidated['dr3_source_id'].notna().sum()}")
print(f"Number of stars with only DR2 source_id: {df_consolidated['dr3_source_id'].isna().sum()}")
print(f"Number of stars with HD Number: {df_consolidated['HD Number'].notna().sum()}")
print(f"Number of stars with GJ Number: {df_consolidated['GJ Number'].notna().sum()}")
print(f"Number of stars with HIP Number: {df_consolidated['HIP Number'].notna().sum()}")


#-------------------------------------------------------------
# Extract data with mass and luminosity information available
#-------------------------------------------------------------
# Filter out stars with missing mass or luminosity information
# df_filtered = df_consolidated[(df_consolidated['Mass [M_Sun]'].notna()) & (df_consolidated['Luminosity [L_Sun]'].notna())]
df_consolidated['Mass [M_Sun]'].replace('', np.nan, inplace=True)
df_consolidated['Luminosity [L_Sun]'].replace('', np.nan, inplace=True)
df_consolidated['Radius [R_Sun]'].replace('', np.nan, inplace=True)
df_consolidated['T_eff [K]'].replace('', np.nan, inplace=True)
df_filtered = df_consolidated.dropna(subset=['Mass [M_Sun]', 'Luminosity [L_Sun]', 'Radius [R_Sun]', 'T_eff [K]'])


# Remove the entry where the effective temperature is larger than 7000K 
# df_filtered = df_filtered[df_filtered['T_eff [K]'] < 7000]

# Save the filtered consolidated data to a new Excel file
df_filtered.to_excel(directory + 'consolidated_results_filtered.xlsx', index=False)
adjust_column_widths(directory + 'consolidated_results_filtered.xlsx')
display(df_filtered)

Original shape of merged_results: (6099, 26)
Shape after removing duplicates: (5925, 26)
Number of rows removed: 174

Number of remaining duplicate dr2_source_id: 1

Example of remaining duplicates:
Empty DataFrame
Columns: [dr3_source_id, RA_dr3, DEC_dr3, Phot G Mean Mag_dr3, Phot BP Mean Mag_dr3, Phot RP Mean Mag_dr3, BP-RP_dr3, Parallax_dr3, T_eff [K]_dr3, Mass [M_Sun]_dr3, Luminosity [L_Sun]_dr3, Radius [R_Sun]_dr3, Spectral Type_dr3, RA_dr2, DEC_dr2, Phot G Mean Mag_dr2, Phot BP Mean Mag_dr2, Phot RP Mean Mag_dr2, BP-RP_dr2, Parallax_dr2, T_eff [K]_dr2, Mass [M_Sun]_dr2, Luminosity [L_Sun]_dr2, Radius [R_Sun]_dr2, Spectral Type_dr2]
Index: []

[0 rows x 25 columns]


Total number of stars: 5925
Number of stars with DR3 source_id: 5925
Number of stars with only DR2 source_id: 0
Number of stars with HD Number: 2437
Number of stars with GJ Number: 1361
Number of stars with HIP Number: 2752


,dr2_source_id,dr3_source_id,RA,DEC,Phot G Mean Mag,Phot BP Mean Mag,Phot RP Mean Mag,BP-RP,Parallax,T_eff [K],Mass [M_Sun],Luminosity [L_Sun],Radius [R_Sun],Spectral Type,HD Number,GJ Number,HIP Number,Object Type
0,NaN,5912380962403300096,263.521562,-59.777571,7.459463,7.753087,6.992328,0.760759,26.695519,5826.763,1.0270743,1.0723715,1.0139297,G,HD 158630,None,HIP 85960,HighPM*
1,NaN,1837182671876057472,304.555931,29.203753,8.813396,9.034739,7.764806,1.269933,29.773776,4862.6016,0.7625022,0.30370355,0.88247246,G,HD 334100A,None,None,HighPM*
2,NaN,1837182676181120896,304.556314,29.203715,9.528364,9.299628,7.864410,1.435218,29.769599,5044.509,0.70530427,0.18704037,0.8509922,G,HD 334100B,None,None,HighPM*
3,NaN,4471238083887731456,270.637685,4.417389,8.704765,9.150189,8.093346,1.056844,25.687915,5015.104,0.7943323,0.39840892,0.83611274,K,HD 164759,None,HIP 88353,HighPM*
4,NaN,4471087175915086080,271.405998,4.655748,6.649316,6.953541,6.166453,0.787088,41.474060,5691.311,1.0329351,0.9404929,0.87734646,G,HD 165401,"GJ 9614, GJ 702.2",HIP 88622,HighPM*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5920,5407414407508423424,5407414407508423680,153.231514,-47.474697,8.085078,8.485525,7.568351,0.917174,26.992809,5409.9487,0.8972189,0.59520715,0.8781961,G,NaN,NaN,NaN,NaN
5921,5407414407508441856,5407414407508442112,153.233454,-47.475572,10.250566,11.039318,9.417773,1.621545,26.910121,4153.005,0.6670827,0.12740755,0.6894917,K,NaN,NaN,NaN,NaN
5922,5114240352186781952,5114240352186781696,55.774785,-12.894209,10.269168,11.106816,9.401341,1.705475,35.218015,4021.2742,0.6024087,0.081752606,0.6964844,K,NaN,NaN,NaN,NaN
5923,4464207428577214464,4464207428577214464,243.327612,13.526227,7.247616,7.659267,6.716437,0.942830,41.259192,5289.4365,0.9276119,0.54569334,0.8102098,G,HD 145958B,"GJ 9548 B, GJ 615.1 B",None,SB*
